In [61]:
import numpy as np
import pandas as pd
import scipy as sp
import math
from scipy.optimize import minimize

In [62]:
# Reading in the asset prices

asset_prices = pd.read_csv("Training Data_Case 3.csv")
asset_prices = asset_prices.drop(columns = asset_prices.columns[0])
asset_prices

,A,B,C,D,E,F,G,H,I,J
0,119.96,198.25,76.54,32.08,53.20,19.47,40.63,43.47,19.76,113.91
1,120.32,201.42,76.73,31.86,53.88,19.23,40.47,45.16,19.88,112.98
2,119.60,200.23,77.60,31.56,55.11,18.93,39.84,46.03,20.29,114.24
3,121.76,199.61,79.48,31.62,55.80,19.15,40.13,46.89,20.01,114.27
4,120.10,199.64,81.49,31.89,55.02,19.16,39.94,47.59,20.12,113.87
...,...,...,...,...,...,...,...,...,...,...
2515,317.31,1385.04,199.14,273.02,52.11,89.45,151.54,390.13,88.25,35.99
2516,317.20,1358.01,202.59,277.45,52.73,90.93,152.39,394.03,87.56,35.89
2517,312.31,1315.00,198.29,281.23,52.71,92.28,153.92,375.70,89.54,36.13
2518,312.45,1319.59,200.78,282.77,52.26,93.49,155.26,368.33,91.36,35.15


In [71]:
# Creating the return dataset

returns = pd.DataFrame(columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

for i in range(0, 2519):
    returns.at[i,'A'] = ((asset_prices.at[i+1,'A'] - asset_prices.at[i,'A']) / asset_prices.at[i,'A'])*100
    returns.at[i,'B'] = ((asset_prices.at[i+1,'B'] - asset_prices.at[i,'B']) / asset_prices.at[i,'B'])*100
    returns.at[i,'C'] = ((asset_prices.at[i+1,'C'] - asset_prices.at[i,'C']) / asset_prices.at[i,'C'])*100
    returns.at[i,'D'] = ((asset_prices.at[i+1,'D'] - asset_prices.at[i,'D']) / asset_prices.at[i,'D'])*100
    returns.at[i,'E'] = ((asset_prices.at[i+1,'E'] - asset_prices.at[i,'E']) / asset_prices.at[i,'E'])*100
    returns.at[i,'F'] = ((asset_prices.at[i+1,'F'] - asset_prices.at[i,'F']) / asset_prices.at[i,'F'])*100
    returns.at[i,'G'] = ((asset_prices.at[i+1,'G'] - asset_prices.at[i,'G']) / asset_prices.at[i,'G'])*100
    returns.at[i,'H'] = ((asset_prices.at[i+1,'H'] - asset_prices.at[i,'H']) / asset_prices.at[i,'H'])*100
    returns.at[i,'I'] = ((asset_prices.at[i+1,'I'] - asset_prices.at[i,'I']) / asset_prices.at[i,'I'])*100
    returns.at[i,'J'] = ((asset_prices.at[i+1,'J'] - asset_prices.at[i,'J']) / asset_prices.at[i,'J'])*100

returns

,A,B,C,D,E,F,G,H,I,J
0,0.3001,1.598991,0.248236,-0.685786,1.278195,-1.232666,-0.393798,3.887739,0.607287,-0.816434
1,-0.598404,-0.590805,1.133846,-0.94162,2.282851,-1.560062,-1.556709,1.926484,2.062374,1.115242
2,1.80602,-0.309644,2.42268,0.190114,1.252041,1.162176,0.727912,1.868347,-1.37999,0.026261
3,-1.363338,0.015029,2.528938,0.85389,-1.397849,0.052219,-0.473461,1.492856,0.549725,-0.350048
4,-1.998335,-0.731316,-0.490858,-0.972092,4.125772,-1.878914,-1.22684,0.0,0.894632,0.184421
...,...,...,...,...,...,...,...,...,...,...
2514,3.331379,2.260746,1.992318,3.115912,-0.420409,5.371657,6.725826,1.429946,1.776035,1.810467
2515,-0.034666,-1.951568,1.73245,1.622592,1.189791,1.654556,0.560908,0.999667,-0.78187,-0.277855
2516,-1.541614,-3.167134,-2.122513,1.362408,-0.037929,1.484659,1.004003,-4.65193,2.261307,0.66871
2517,0.044827,0.349049,1.255737,0.547594,-0.853728,1.311227,0.870582,-1.961672,2.032611,-2.712427


In [72]:
# Converting the type of data in the returns dataset 

stocks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

for s in stocks:
    returns.loc[:, s] = pd.to_numeric(returns.loc[:,s])

returns.dtypes

A    float64
B    float64
C    float64
D    float64
E    float64
F    float64
G    float64
H    float64
I    float64
J    float64
dtype: object

In [65]:
# Creating the training and testing data (to test this optimization method)

training_data = returns.loc[:2000,:].copy() 
testing_data = returns.loc[2001:, :].copy()

testing_data

,A,B,C,D,E,F,G,H,I,J
2001,-0.117199,-0.271862,-0.892972,-0.312500,2.087091,0.176488,0.760985,0.553843,0.319524,0.351827
2002,-0.858023,-0.622444,-1.413107,-1.174757,2.725896,2.711168,-0.828325,3.200831,-1.153213,1.564186
2003,2.076090,0.325515,0.414228,1.066966,1.007371,1.429388,1.547658,-1.238608,-1.966667,0.292087
2004,-0.647359,-0.914137,-0.923258,1.220556,1.021649,0.112740,1.048303,2.880449,0.997393,2.647604
2005,-1.205903,0.166486,-0.317230,-1.105613,1.516976,-1.210586,-0.079802,-7.006938,1.492537,0.928553
...,...,...,...,...,...,...,...,...,...,...
2514,3.331379,2.260746,1.992318,3.115912,-0.420409,5.371657,6.725826,1.429946,1.776035,1.810467
2515,-0.034666,-1.951568,1.732450,1.622592,1.189791,1.654556,0.560908,0.999667,-0.781870,-0.277855
2516,-1.541614,-3.167134,-2.122513,1.362408,-0.037929,1.484659,1.004003,-4.651930,2.261307,0.668710
2517,0.044827,0.349049,1.255737,0.547594,-0.853728,1.311227,0.870582,-1.961672,2.032611,-2.712427


In [66]:
# Calculating the covariance matrix for the training data

cov_matrix = pd.DataFrame(np.cov(np.transpose(training_data)))
cov_matrix

,0,1,2,3,4,5,6,7,8,9
0,3.318465,0.944092,0.018716,2.755643,0.160106,3.744737,3.111523,0.040775,0.793020,0.151374
1,0.944092,2.029488,0.375181,0.805215,-0.139516,0.662320,0.810235,-0.061375,0.847454,-0.284699
2,0.018716,0.375181,1.701872,-0.105901,0.726190,-0.307837,-0.157009,-0.114446,0.185223,0.854720
3,2.755643,0.805215,-0.105901,3.739152,0.273135,4.190378,3.422478,0.030327,0.671619,0.250516
4,0.160106,-0.139516,0.726190,0.273135,3.564077,0.545452,0.374200,0.025906,-0.117626,2.992740
5,3.744737,0.662320,-0.307837,4.190378,0.545452,7.198542,4.860596,0.110543,0.614229,0.569758
6,3.111523,0.810235,-0.157009,3.422478,0.374200,4.860596,4.347519,0.120853,0.728569,0.347029
7,0.040775,-0.061375,-0.114446,0.030327,0.025906,0.110543,0.120853,8.797616,-0.028797,0.134912
8,0.793020,0.847454,0.185223,0.671619,-0.117626,0.614229,0.728569,-0.028797,2.074880,-0.138665
9,0.151374,-0.284699,0.854720,0.250516,2.992740,0.569758,0.347029,0.134912,-0.138665,5.501916


In [67]:
# Finding the weight vector of the last day in the dataset (Just for testing the optimization algorithm)

initial_weights = [0.1 for i in range(0, 10)]
last_return = returns.loc[2518, :]
initial_weights = np.array(initial_weights)
last_return = np.array(last_return)

def portfolio_objective(weights, returns, cov_matrix):
    portfolio_return = np.dot(weights, last_return)
    portfolio_risk = np.matmul(np.matmul(weights, cov_matrix), np.transpose(weights))
    return -(portfolio_return/math.sqrt(portfolio_risk))

constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
               {'type': 'ineq', 'fun': lambda x: x}]

result = minimize(portfolio_objective, initial_weights, args=(returns, cov_matrix), method='SLSQP', bounds=[(0, 1)] * returns.shape[1], constraints=constraints)
optimized_weights = result.x
print(optimized_weights)
print(result)
    

[0.         0.         0.59013415 0.30270926 0.06080337 0.
 0.         0.         0.04635322 0.        ]
     fun: -1.8856960008346297
     jac: array([ 1.17101881e+00,  1.40835749e+00, -9.58144665e-06,  2.04145908e-05,
       -5.79655170e-06,  1.44954526e+00,  1.29048710e+00,  4.10099131e+00,
       -3.14414501e-06,  3.00594091e+00])
 message: 'Optimization terminated successfully'
    nfev: 100
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.        , 0.        , 0.59013415, 0.30270926, 0.06080337,
       0.        , 0.        , 0.        , 0.04635322, 0.        ])


In [68]:
# Function for returning optimal portfolio weights

def return_weights(return_vector, cov_matrix):
    
    initial_weights = [0.1 for i in range(0, 10)]
    initial_weights = np.array(initial_weights)
    return_vector = np.array(return_vector)
    
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
               {'type': 'ineq', 'fun': lambda x: x}]
    
    result = minimize(portfolio_objective, initial_weights, args=(returns, cov_matrix), method='SLSQP', bounds=[(0, 1)] * returns.shape[1], constraints=constraints)
    optimized_weights = result.x
    
    # print("The optimized weights are: ", optimized_weights)
    print("The sharpe ratio is: ", result.fun)
    
    return optimized_weights

In [69]:
# Testing the optimization algorithm using the testing data

for i in range(2001,2519):
    training_data.loc[len(training_data.index)] = testing_data.loc[i,:]
    cov_matrix = pd.DataFrame(np.cov(np.transpose(training_data)))
    return_weights(testing_data.loc[i,:],cov_matrix)
    
# We are consistently getting a sharpe ratio of around 1.92, which is a great sign (but we can still try to improve the sharpe ratio)

The sharpe ratio is:  -1.8860256393190498
The sharpe ratio is:  -1.8858823072741322
The sharpe ratio is:  -1.8862505422270188
The sharpe ratio is:  -1.886712762295511
The sharpe ratio is:  -1.8870977025840603
The sharpe ratio is:  -1.8875310612696847
The sharpe ratio is:  -1.8856197239504648
The sharpe ratio is:  -1.8860318484768481
The sharpe ratio is:  -1.8863262551177933
The sharpe ratio is:  -1.8863353826635718
The sharpe ratio is:  -1.884928937747417
The sharpe ratio is:  -1.8851082695900683
The sharpe ratio is:  -1.8849222480905419
The sharpe ratio is:  -1.8853845255433535
The sharpe ratio is:  -1.8854498548438248
The sharpe ratio is:  -1.8857451152349423
The sharpe ratio is:  -1.8862090674596574
The sharpe ratio is:  -1.8864009968416595
The sharpe ratio is:  -1.8868419813969786
The sharpe ratio is:  -1.8870248510832133
The sharpe ratio is:  -1.8864596834433685
The sharpe ratio is:  -1.8858603680479413
The sharpe ratio is:  -1.8862608784193176
The sharpe ratio is:  -1.88626687602

In [92]:
# The main function that UChicago will be running

def allocate_portfolio(asset_prices):
    
    stock_returns = returns.copy()
    asset_prices = np.array(asset_prices)
    stock_returns.loc[len(stock_returns.index)] = asset_prices
    cov_matrix = pd.DataFrame(np.cov(np.transpose(stock_returns)))
    optimized_weights = return_weights(testing_data.loc[i,:],cov_matrix)
    
    return optimized_weights

# Testing the function above

allocate_portfolio([317, 1310, 210, 296, 53, 96, 158, 341, 94, 33])


The sharpe ratio is:  -0.6456981277552652


array([0.        , 0.        , 0.07602904, 0.        , 0.87001728,
       0.        , 0.        , 0.        , 0.05395368, 0.        ])